In [25]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

In [26]:
#https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541

def get_features(gray_image):
    "Detecting features(corners) using Shi-Thomasi Corner Detectors"
    points = cv2.goodFeaturesToTrack(gray_image, mask=None, maxCorners=100,
                                     qualityLevel=0.1, minDistance=10, blockSize=5)
    return points

In [27]:
#https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323

def LK_model(gray_t_1, gray_t, points_t_1):
    "Detecting Optical Flow using Lucas Kanade Sparse Optical FLow method"
    points, status, error = cv2.calcOpticalFlowPyrLK(gray_t_1,gray_t,points_t_1,None,
                                                    winSize=(10,10),maxLevel=2, flags=10, minEigThreshold=0.03)
    return points, status, error

In [31]:
video = cv2.VideoCapture("Optical_Flow/traffic_1639723780.mp4")

#checking fps
fps = video.get(5)
print('Frames per second : ', fps,'FPS')

#checking total number of frames
frame_count = video.get(7)
print('Frame count : ', frame_count)

#obtaining frame size information
frame_width = int(video.get(3))
frame_height = int(video.get(4))
frame_size = (frame_width,frame_height)
print("Shape of the image",frame_size)

#Reading first_frame
ret, prev_frame = video.read()

#covnerting to gray image and finding the keypoints
prev_gray_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_points = get_features(prev_gray_frame)
mask_image = np.zeros_like(prev_gray_frame)

#coloring the flow lines
color = np.random.randint(0,255,(100,3))

#writing optical flow images into a video file
output = cv2.VideoWriter('Optical_Flow/optical_flow.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, frame_size)

while(video.isOpened()):
    ret, current_frame = video.read()
    if ret == True:
        current_gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
        current_points, status, error = LK_model(prev_gray_frame, current_gray_frame, prev_points)
        
        if current_points is not None:
            current_points = current_points[status==1]
            old_points = prev_points[status==1]
        
        for i,(current,prev) in enumerate(zip(current_points, old_points)):
            a,b = current.ravel()
            c,d = prev.ravel()
            mask_image = cv2.line(mask_image, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            current_frame = cv2.circle(current_frame,(int(a),int(b)), 5, color[i].tolist(), -1)
        out = cv2.add(current_frame, mask_image)
        output.write(out)
        cv2.imshow("Frame",out)
        k = cv2.waitKey(20)
        if k ==27:
            break
        
        #setting the current frame as previous frame for next iteration
        prev_gray_frame = current_gray_frame.copy()
        prev_points = current_points.reshape(-1,1,2)
    else:
        break
    
video.release()
cv2.destroyAllWindows()

Frames per second :  30.0 FPS
Frame count :  400.0
Shape of the image (640, 360)
